# Final Project: Fake News Detection

By Felix Daubner - Hochschule der Medien

Module 'Supervised and Unsupervised Learning' - Prof. Dr.-Ing. Johannes Maucher

## Model Training

To-Do:
- Define model architecture

In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow import keras
import pickle

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

NUM_WORDS=3000
MAX_SEQUENCE_LEN = 57

Feature Selection
- nur Text
- Kategorien
- alles

Performance vergleichen

- CNN
- Feedforward NN
- LSTM
- Bidirectional LSTM

This section contains the model training and feature selection. Different types of models should be trained and then compared to find out which model fits the challenge, to determine whether a political statement was fake-news or true, best. There are three types of models to be compared: MLP, CNN and LSTM. Those models should also vary in terms of hyperparameters like layers, neurons, optimization and else. The best model is evaluated and then optimized in the next section.

### Prepare data for training and validation

In [22]:
data = pd.read_json("data/processed.json", orient="records", lines=True)

Before starting defining the different models, the data is prepared for the training process. The neural network to be trained only takes numpy arrays as input. Thus, the data currently saved as a pandas DataFrame is converted in to a numpy array. In this conversion process, only "token", the encoded channel and issue columns and "truth" are kept meaning column 'statement' is dropped as it can not be used for model training.

In [23]:
X = np.array(data.drop(["statement", "statement_stop", "token_stop"], axis=1).apply(np.array))
y = np.array(data["truth"])

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=42)

### Prepare data for testing

In [33]:
test = pd.read_json("data/LIAR_processed.json", orient="records", lines=True)

In [34]:
X_test = np.array(test.drop(["statement", "statement_stop", "token_stop"], axis=1).apply(np.array))
y_test = np.array(test["truth"])

In the following, four different types models are trained and evaluated. Based on those evaluations, the best model is chosen and optimized until pre-defined metrics reach their peak.

When looking at fake-news detection, it is decided which of accuracy, precision or recall should be optimized. Most often, accuracy is not a good metric as it doesn't include the cost of mis-predicting. That's why either precision or recall should be used.
The worst case at fake-news is when a fake-news is not identified as fake-news. Whereas the other way, a true statement being classified as fake-news does not harm in the same way. Translating this into the terms of this project means a false positive ("a statement which is 'fake' (0) gets classified as 'true' (1)") is worse than a false negative ("a statement which is 'true' (1) gets classified as 'false' (0)"). The metrics focusing on optimizing the false positives is precision.

### Feedforward Neural Network

### Convolutional Neural Network

### Long Short-Term Memory

### Bi-directional Long Short-Term Memory